In [26]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [25]:
!pip uninstall tensorflow

^C


In [27]:
!pip install keras scikeras

In [29]:
!pip install distutils

ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [33]:
!pip install --upgrade setuptools


  Using cached setuptools-75.2.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.2.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.0.4
    Uninstalling setuptools-58.0.4:
      Successfully uninstalled setuptools-58.0.4


In [1]:
import tensorflow

In [2]:
tensorflow.__version__

'2.16.1'

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
import mlflow
import mlflow.tensorflow

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, GRU, Dropout

In [5]:
# Load preprocessed data
pre_fraud_data_df = pd.read_csv('../data/preprocessed_fraud_data.csv')

In [6]:
pre_fraud_data_df

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,89215,2,0,1,39,732758368,0,84,1,0.000008,2,5
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,24078,0,0,0,53,350311387,0,171,1,0.000892,1,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,131216,2,3,1,53,2621473820,1,171,1,15.000000,18,3
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,3977,2,4,1,41,3840542443,0,181,1,0.000089,13,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,68757,0,4,1,45,415583117,0,171,1,0.000009,18,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,2015-03-29 00:30:47,43,125335,2,0,1,28,3451154526,1,171,1,0.000008,0,6
151108,274471,2015-05-15 17:43:29,2015-05-26 12:24:39,35,63001,2,4,1,32,2439047221,0,118,1,0.000038,12,1
151109,368416,2015-03-03 23:07:31,2015-05-20 07:07:47,40,64204,2,2,0,26,2748470523,0,84,1,0.000006,7,2
151110,207709,2015-07-09 20:06:07,2015-09-07 09:34:46,46,13118,2,0,1,37,3601174708,0,171,1,0.000009,9,0


In [7]:
#Function to extract datetime features month and year 
def extract_datetime_features(df, datetime_column):
    df_copy = df.copy()
    df_copy[datetime_column] = pd.to_datetime(df_copy[datetime_column])
    df_copy['year'] = df_copy[datetime_column].dt.year
    df_copy['month'] = df_copy[datetime_column].dt.month
    df_copy = df_copy.drop(columns=[datetime_column])
    return df_copy
datetime_column = 'purchase_time' 
fraud_data_df = extract_datetime_features(pre_fraud_data_df, datetime_column)

In [8]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,2015-02-24 22:55:49,34,89215,2,0,1,39,732758368,0,84,1,0.000008,2,5,2015,4
1,333320,2015-06-07 20:39:50,16,24078,0,0,0,53,350311387,0,171,1,0.000892,1,0,2015,6
2,1359,2015-01-01 18:52:44,15,131216,2,3,1,53,2621473820,1,171,1,15.000000,18,3,2015,1
3,150084,2015-04-28 21:13:25,44,3977,2,4,1,41,3840542443,0,181,1,0.000089,13,0,2015,5
4,221365,2015-07-21 07:09:52,39,68757,0,4,1,45,415583117,0,171,1,0.000009,18,2,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,43,125335,2,0,1,28,3451154526,1,171,1,0.000008,0,6,2015,3
151108,274471,2015-05-15 17:43:29,35,63001,2,4,1,32,2439047221,0,118,1,0.000038,12,1,2015,5
151109,368416,2015-03-03 23:07:31,40,64204,2,2,0,26,2748470523,0,84,1,0.000006,7,2,2015,5
151110,207709,2015-07-09 20:06:07,46,13118,2,0,1,37,3601174708,0,171,1,0.000009,9,0,2015,9


In [9]:
fraud_data_df .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                151112 non-null  int64  
 1   signup_time            151112 non-null  object 
 2   purchase_value         151112 non-null  int64  
 3   device_id              151112 non-null  int64  
 4   source                 151112 non-null  int64  
 5   browser                151112 non-null  int64  
 6   sex                    151112 non-null  int64  
 7   age                    151112 non-null  int64  
 8   ip_address             151112 non-null  int64  
 9   class                  151112 non-null  int64  
 10  country                151112 non-null  int64  
 11  transaction_frequency  151112 non-null  int64  
 12  transaction_velocity   151112 non-null  float64
 13  hour_of_day            151112 non-null  int64  
 14  day_of_week            151112 non-nu

In [10]:
# Convert 'year' and 'month' columns to int64
fraud_data_df['year'] = fraud_data_df['year'].astype('int64')
fraud_data_df['month'] = fraud_data_df['month'].astype('int64')
fraud_data_df['transaction_velocity'] = fraud_data_df['transaction_velocity'].astype('int64')

In [11]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,2015-02-24 22:55:49,34,89215,2,0,1,39,732758368,0,84,1,0,2,5,2015,4
1,333320,2015-06-07 20:39:50,16,24078,0,0,0,53,350311387,0,171,1,0,1,0,2015,6
2,1359,2015-01-01 18:52:44,15,131216,2,3,1,53,2621473820,1,171,1,15,18,3,2015,1
3,150084,2015-04-28 21:13:25,44,3977,2,4,1,41,3840542443,0,181,1,0,13,0,2015,5
4,221365,2015-07-21 07:09:52,39,68757,0,4,1,45,415583117,0,171,1,0,18,2,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,2015-01-27 03:03:34,43,125335,2,0,1,28,3451154526,1,171,1,0,0,6,2015,3
151108,274471,2015-05-15 17:43:29,35,63001,2,4,1,32,2439047221,0,118,1,0,12,1,2015,5
151109,368416,2015-03-03 23:07:31,40,64204,2,2,0,26,2748470523,0,84,1,0,7,2,2015,5
151110,207709,2015-07-09 20:06:07,46,13118,2,0,1,37,3601174708,0,171,1,0,9,0,2015,9


In [12]:
# Encode Categorical Features
def encode_categorical_features(df):
    le = LabelEncoder()
    for column in df.select_dtypes(include=['object']).columns:
        df[column] = le.fit_transform(df[column])
    return df
fraud_data_df = encode_categorical_features(fraud_data_df)

In [11]:
fraud_data_df

,user_id,signup_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country,transaction_frequency,transaction_velocity,hour_of_day,day_of_week,year,month
0,22058,41956,34,89215,2,0,1,39,732758368,0,84,1,0,2,5,2015,4
1,333320,106645,16,24078,0,0,0,53,350311387,0,171,1,0,1,0,2015,6
2,1359,971,15,131216,2,3,1,53,2621473820,1,171,1,15,18,3,2015,1
3,150084,81672,44,3977,2,4,1,41,3840542443,0,181,1,0,13,0,2015,5
4,221365,133741,39,68757,0,4,1,45,415583117,0,171,1,0,18,2,2015,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151107,345170,23950,43,125335,2,0,1,28,3451154526,1,171,1,0,0,6,2015,3
151108,274471,92134,35,63001,2,4,1,32,2439047221,0,118,1,0,12,1,2015,5
151109,368416,46366,40,64204,2,2,0,26,2748470523,0,84,1,0,7,2,2015,5
151110,207709,126488,46,13118,2,0,1,37,3601174708,0,171,1,0,9,0,2015,9


In [13]:
cleaned_fraud_data = pd.read_csv('../data/cleaned_Fraud_Data.csv')

In [23]:
cleaned_fraud_data = cleaned_fraud_data.fillna(0)

In [24]:
#Define functions for model building and evaluation

def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [25]:
def build_rnn_model(input_shape):
    model = Sequential()
    model.add(GRU(units=64, return_sequences=True, input_shape=input_shape))
    model.add(GRU(units=32))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [26]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=32))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [27]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name, experiment_name):
    # MLflow tracking
    mlflow.set_tracking_uri('http://localhost:5000')  # Set your MLflow tracking server
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run():
        mlflow.log_param('model', model_name)

        # Compile the model
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

        # Evaluate the model
        y_pred = model.predict(X_test)
        y_pred_binary = (y_pred > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary)
        recall = recall_score(y_test, y_pred_binary)
        f1 = f1_score(y_test, y_pred_binary)
        roc_auc = roc_auc_score(y_test, y_pred)

        # Log metrics to MLflow
        mlflow.log_metric('accuracy', accuracy)
        mlflow.log_metric('precision', precision)
        mlflow.log_metric('recall', recall)
        mlflow.log_metric('f1_score', f1)
        mlflow.log_metric('roc_auc', roc_auc)

        # Log model to MLflow
        mlflow.tensorflow.log_model(model, artifact_path='model')
        print(f'Model: {model_name}')
        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1 Score: {f1:.4f}')
        print(f'ROC AUC: {roc_auc:.4f}')

In [28]:
# Data Preparation
# Fraud Data
X_fraud = cleaned_fraud_data.drop('class', axis=1)
y_fraud = cleaned_fraud_data['class']
X_fraud_train, X_fraud_test, y_fraud_train, y_fraud_test = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

In [29]:
X_fraud_train = X_fraud_train.values.reshape(X_fraud_train.shape[0], X_fraud_train.shape[1], 1)
X_fraud_test = X_fraud_test.values.reshape(X_fraud_test.shape[0], X_fraud_test.shape[1], 1)

### Run "mlflow ui" in the bash then calculate 

In [21]:
# Fraud Data
print("Fraud Data - CNN")
cnn_model_fraud = build_cnn_model(X_fraud_train.shape[1:])
train_and_evaluate_model(cnn_model_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, 'CNN', 'Fraud Detection')

Fraud Data - CNN


2024/10/21 23:10:17 INFO mlflow.tracking.fluent: Experiment with name 'Fraud Detection' does not exist. Creating a new experiment.


Epoch 1/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.8261 - loss: 2494395.7500 - val_accuracy: 0.9062 - val_loss: nan
Epoch 2/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8320 - loss: 1144841.8750 - val_accuracy: 0.9061 - val_loss: nan
Epoch 3/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8342 - loss: 1038558.3750 - val_accuracy: 0.9061 - val_loss: nan
Epoch 4/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8373 - loss: 1085264.8750 - val_accuracy: 0.9061 - val_loss: nan
Epoch 5/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8371 - loss: 899952.5000 - val_accuracy: 0.9043 - val_loss: nan
594/594 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step


2024/10/21 23:11:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/21 23:12:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/21 23:12:29 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-doe-670 at: http://localhost:5000/#/experiments/815254548227489244/runs/25c7d379d9504e83830cbdcf5a11e42f.
2024/10/21 23:12:29 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/815254548227489244.


Model: CNN
Accuracy: 0.9043
Precision: 0.3767
Recall: 0.0308
F1 Score: 0.0570
ROC AUC: 0.5128


In [30]:
print("Fraud Data - LSTM")
lstm_model_fraud = build_lstm_model(X_fraud_train.shape[1:])
train_and_evaluate_model(lstm_model_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, 'LSTM', 'Fraud Detection')

Fraud Data - LSTM


c:\Users\Hello\Desktop\Html Tutorial\Document\KAIM Courses\Week-08-9_Adey_Innovations\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 67s 25ms/step - accuracy: 0.9401 - loss: 0.2159 - val_accuracy: 0.9559 - val_loss: 0.1788
Epoch 2/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 65s 27ms/step - accuracy: 0.9563 - loss: 0.1779 - val_accuracy: 0.9559 - val_loss: 0.1789
Epoch 3/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 65s 28ms/step - accuracy: 0.9573 - loss: 0.1745 - val_accuracy: 0.9559 - val_loss: 0.1793
Epoch 4/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 58s 25ms/step - accuracy: 0.9558 - loss: 0.1792 - val_accuracy: 0.9559 - val_loss: 0.1786
Epoch 5/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 64s 27ms/step - accuracy: 0.9559 - loss: 0.1787 - val_accuracy: 0.9559 - val_loss: 0.1786
594/594 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step


2024/10/21 23:26:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/21 23:26:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/21 23:26:31 INFO mlflow.tracking._tracking_service.client: 🏃 View run resilient-cod-726 at: http://localhost:5000/#/experiments/815254548227489244/runs/dabbed58c3874ad490d2f7e8819f0845.
2024/10/21 23:26:31 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/815254548227489244.


Model: LSTM
Accuracy: 0.9559
Precision: 1.0000
Recall: 0.5294
F1 Score: 0.6923
ROC AUC: 0.7551


In [31]:
print("Fraud Data - RNN")
rnn_model_fraud = build_rnn_model(X_fraud_train.shape[1:])
train_and_evaluate_model(rnn_model_fraud, X_fraud_train, y_fraud_train, X_fraud_test, y_fraud_test, 'RNN', 'Fraud Detection')

Fraud Data - RNN


c:\Users\Hello\Desktop\Html Tutorial\Document\KAIM Courses\Week-08-9_Adey_Innovations\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 86s 30ms/step - accuracy: 0.9491 - loss: 0.1965 - val_accuracy: 0.9559 - val_loss: 0.1792
Epoch 2/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 80s 34ms/step - accuracy: 0.9566 - loss: 0.1764 - val_accuracy: 0.9559 - val_loss: 0.1788
Epoch 3/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 83s 35ms/step - accuracy: 0.9576 - loss: 0.1737 - val_accuracy: 0.9559 - val_loss: 0.1790
Epoch 4/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 71s 30ms/step - accuracy: 0.9557 - loss: 0.1793 - val_accuracy: 0.9559 - val_loss: 0.1789
Epoch 5/5
2376/2376 ━━━━━━━━━━━━━━━━━━━━ 71s 30ms/step - accuracy: 0.9564 - loss: 0.1771 - val_accuracy: 0.9559 - val_loss: 0.1787
594/594 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step


2024/10/21 23:34:46 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/21 23:35:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/21 23:35:02 INFO mlflow.tracking._tracking_service.client: 🏃 View run mysterious-flea-914 at: http://localhost:5000/#/experiments/815254548227489244/runs/08160f69df0147ba9edacbe29ccd278f.
2024/10/21 23:35:02 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/815254548227489244.


Model: RNN
Accuracy: 0.9559
Precision: 1.0000
Recall: 0.5294
F1 Score: 0.6923
ROC AUC: 0.7692
